In [1]:
import numpy as np
import pandas as pd

# Bloomberg

In [2]:
data = pd.read_csv('temp2.csv', index_col=0)

In [3]:
data

,USD,EUR,JPY,GBP,CHF,CAD,AUD,HKD
CUR,,,,,,,,
USD,1.0000,1.0889,0.0084,1.4879,1.0400,0.7928,0.7752,0.1290
EUR,0.9183,1.0000,0.0077,1.3661,0.9548,0.7279,0.7116,0.1184
JPY,119.1300,129.7300,1.0000,177.3680,123.8960,94.4740,92.3510,15.3635
GBP,0.6724,0.7321,0.0056,1.0000,0.6992,0.5330,0.5211,0.0867
CHF,0.9618,1.0471,0.0081,1.4314,1.0000,0.7629,0.7471,0.1240
CAD,1.2609,1.3732,0.0106,1.8757,1.3117,1.0000,0.9775,0.1627
AUD,1.2907,1.4046,0.0108,1.9197,1.3421,1.0231,1.0000,0.1664
HKD,7.7535,8.4429,0.0651,11.5325,8.0629,6.1464,6.0079,1.0000


In [4]:
currencies = data.columns

In [5]:
list(currencies)

['USD', 'EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'HKD']

In [6]:
curr = 'EUR'
value = 100
path = ['EUR']
best_path = []
best_value = 100
path_choice = list(currencies)

for i in range(10000):
    next_curr = np.random.choice(path_choice)
    exchange = data.ix[curr][next_curr]    
    value *= exchange
    curr = next_curr
    path.append(next_curr)
    path_choice.remove(next_curr)
    
    if next_curr == 'EUR':
        if value > best_value:
            best_value = value
            best_path = path
            print best_value - 100, best_path
        value = 100
        path = ['EUR']
        path_choice = list(currencies)

0.143957748857 ['EUR', 'CAD', 'CHF', 'GBP', 'AUD', 'USD', 'HKD', 'JPY', 'EUR']
0.1944834968 ['EUR', 'CHF', 'AUD', 'EUR']
0.36879776696 ['EUR', 'CHF', 'JPY', 'USD', 'GBP', 'CAD', 'HKD', 'EUR']
0.437880337728 ['EUR', 'CAD', 'HKD', 'AUD', 'CHF', 'JPY', 'GBP', 'EUR']
0.447155962982 ['EUR', 'HKD', 'CHF', 'JPY', 'GBP', 'AUD', 'USD', 'EUR']
0.472038691736 ['EUR', 'CAD', 'HKD', 'CHF', 'JPY', 'GBP', 'AUD', 'USD', 'EUR']


## BTC-e

In [7]:
pairs = list(pd.read_json("https://btc-e.com/api/3/info").index)
data = pd.read_json("https://btc-e.com/api/3/ticker/" + "-".join(pairs))
currs = [pair.split("_") for pair in pairs]
currs = list(set(sum(currs, [])))

In [8]:
exchanges = pd.DataFrame(index=currs, columns=currs)
for pair in pairs:
    from_curr, to_curr = pair.split("_")
    exchanges.ix[from_curr][to_curr] = data[pair]['sell'] * 0.998
    exchanges.ix[to_curr][from_curr] =  (1 / data[pair]['buy']) * 0.998
    
exchanges

,ppc,usd,btc,ltc,nvc,nmc,gbp,rur,cnh,eur
ppc,NaN,0.22954,0.00098802,NaN,NaN,NaN,NaN,NaN,NaN,NaN
usd,4.320346,NaN,0.004355302,0.7063491,2.310185,3.042683,0.6634315,51.88603,6.390194,0.9423629
btc,998,228.566,NaN,162.013,522.5131,697.9021,150.7882,11902.21,1457.09,214.5946
ltc,NaN,1.405184,0.0061377,NaN,NaN,NaN,0.931134,73.06358,8.96204,1.321352
nvc,NaN,0.425148,0.00187624,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nmc,NaN,0.32435,0.00140718,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gbp,NaN,1.485423,0.006410054,1.040667,NaN,NaN,NaN,NaN,NaN,NaN
rur,NaN,0.01906763,8.316667e-05,0.0134156,NaN,NaN,NaN,NaN,NaN,0.01798522
cnh,NaN,0.1549714,0.0006734234,0.1108889,NaN,NaN,NaN,NaN,NaN,NaN
eur,NaN,1.054866,0.004615841,0.7481259,NaN,NaN,NaN,54.50079,NaN,NaN


In [9]:
best_path = []
best_value = 0

for i in range(1000):
    
    value = 100
    starting_curr = np.random.choice(currs)
    
    path = [starting_curr]
    curr = starting_curr        
    next_curr = None    
    
    while next_curr != starting_curr:        
        next_curr = np.random.choice(exchanges.ix[curr].dropna().index)
        rate = exchanges.ix[curr][next_curr]
        value *= rate
        curr = next_curr
        path.append(next_curr)
    
    if value > best_value:
        best_value = value
        best_path = path
        
        print value - 100, path    

-3.5023235049 [u'rur', u'usd', u'eur', u'ltc', u'btc', u'eur', u'rur']
-2.65826342459 [u'usd', u'nvc', u'btc', u'ltc', u'rur', u'eur', u'usd']
-1.87615217209 [u'btc', u'cnh', u'btc']
-0.228344304505 [u'usd', u'btc', u'ppc', u'usd']


# Bitfinex

In [10]:
pairs = list(pd.read_json("https://api.bitfinex.com/v1/symbols")[0])
currs = [[a[:3], a[3:]] for a in pairs]
currs = list(set(sum(currs, [])))
# data = pd.read_json("https://api.bitfinex.com/v1/pubticker/" + "-".join(pairs))

In [11]:
exchanges = pd.DataFrame(index=currs, columns=currs)
for pair in pairs:
    data = pd.read_json("https://api.bitfinex.com/v1/pubticker/" + pair,typ='series', orient='records')
    from_curr, to_curr = pair[:3], pair[3:]
    exchanges.ix[from_curr][to_curr] = data['bid'] * 0.998
    exchanges.ix[to_curr][from_curr] =  (1 / data['ask']) * 0.998
    
exchanges

,ltc,drk,usd,btc
ltc,NaN,NaN,1.419356,0.00614768
drk,NaN,NaN,3.224837,0.01394206
usd,0.6989774,0.3068032,NaN,0.004310457
btc,161.9867,71.17387,231.0071,NaN


In [12]:
best_path = []
best_value = 0

for i in range(1000):
    
    value = 100
    starting_curr = np.random.choice(currs)
    
    path = [starting_curr]
    curr = starting_curr        
    next_curr = None    
    
    while next_curr != starting_curr:        
        next_curr = np.random.choice(exchanges.ix[curr].dropna().index)
        rate = exchanges.ix[curr][next_curr]
        value *= rate
        curr = next_curr
        path.append(next_curr)
    
    if value > best_value:
        best_value = value
        best_path = path
        
        print value - 100, path    

-0.790244516039 [u'ltc', u'usd', u'ltc']
-0.415766271709 [u'btc', u'ltc', u'btc']


In [13]:
100 * 0.003952413 * 253 * 253

25299.0003717